<font size="1">Copyright 2021, by the California Institute of Technology. ALL RIGHTS RESERVED. United States Government sponsorship acknowledged. Any commercial use must be negotiated with the Office of Technology Transfer at the California Institute of Technology.</font>
    
<font size="1">This software may be subject to U.S. export control laws and regulations. By accepting this document, the user agrees to comply with all applicable U.S. export laws and regulations. User has the responsibility to obtain export licenses, or other export authority as may be required, before exporting such information to foreign countries or providing access to foreign persons.<font>

# Processing TopsApp

## Introduction
In this notebook, we will run the various steps of processing with topsApp.py. 

topsApp.py is a pair-by-pair interferometric processor that takes as input two Sentinel-1 SAR acquisitions acquired in TOPS mode.At this time, topsApp only supports SLC data from Sentinel-1 A and B. Processing is supported across the Sentinel-1 constellation, i.e. data acquired from A and B can be combined**

Processing of TopsApp involves:

**Downloading Inputs**
  - Downloading SLCs : Both Reference and Secondary
  - Downlaoding DEMs : Based on supplied region of interest (min/max lat/lon)
  - Downloading Orbits : Based on SLC dates

**Processing with ISCE**
  - Creating ISCE input configuration files : topsApp.xml, reference.xml, secondary.xml
  - Steps of topsApp processing (in order):
     - startup
     - preprocess
     - computeBaselines**
     - verifyDEM
     - topo
     - subsetoverlaps
     - coarseoffsets
     - coarseresamp
     - overlapifg
     - prepesd
     - esd
     - rangecoreg
     - fineoffsets
     - fineresamp
     - ion
     - burstifg
     - mergebursts
     - filter
     - unwrap
     - unwrap2stage
     - geocode
     - denseoffsets
     - filteroffsets
     - geocodeoffsets
     
     The steps of topsApp depends on start and end parameter supplied while calling topsApp. For example, the following command will process from the first step (startup) till step 'geocode'
    ```
     /opt/isce2/isce/applications/topsApp.py --start=startup --end=geocode
    ```
    
    Also, some steps can be turned on or off by setting some properties in topsApp.xml. For example, 'unwrap' step will be ignored if'do_unwrap' property value is set to False
   


# Configurable Parameters

We can run this notebook to process topsApp with many different combinations of SLCs and region of interest as well as with different combination of topsApp properties like swaths, azimuth looks, range looks etc. In the following section we initizale all these variables.

**Input Parameters**

- **min_lat, max_lat, min_loc, max_loc**: min/max latitude and longditude of Region of Interest.
    Example:
        min_lat = 31.9 # type: number
- **reference_slcs, secondary_slcs** : List of reference SLCs and secondary SLCs.
    Example:
        reference_slcs: List[str] = ["S1B_IW_SLC__1SDV_20190628T014909_20190628T014936_016890_01FC87_55C8"]
        
**Tunable Parameters**
- **swaths** : array conntaining swath values to be considerate.
    Example:
        swaths: List[int] = [3]
- **range_looks** : range looks value. Number.
    Example: 
        range_looks = 7        
- **azimuth_looks** : azimuth looks value. Number.
    Example:
        azimuth_looks = 3
- **do_unwrap** : True or False if unwrapping shoud be done or not.
    Example:
        do_unwrap = "True"
- **unwrapper_name** : Unwrapper name when do_unwrap is True.
    Example:
        unwrapper_name = "snaphu_mcf"
- **do_denseoffsets** : True/False for denseoffsets processing.
    Example:
        do_denseoffsets = "False"


In [ ]:
from typing import List
min_lat = 31.9 # type: number
max_lat = 33.94 # type: number
min_lon =  -118.74 # type: number
max_lon = -115.69 # type: number
        
reference_slcs: List[str] = ["S1B_IW_SLC__1SDV_20190628T014909_20190628T014936_016890_01FC87_55C8"]
secondary_slcs: List[str] = ["S1B_IW_SLC__1SDV_20190710T014909_20190710T014936_017065_0201B8_0252"]                     
    
sensor_name = "SENTINEL1"
swaths: List[int] = [1, 2, 3]
range_looks = 7
azimuth_looks = 3
do_unwrap = "True"
unwrapper_name = "snaphu_mcf"
do_denseoffsets = "False"

## Setup output directories

In [ ]:
import os

# directory in which the notebook resides
if 'tutorial_home_dir' not in globals():
    tutorial_home_dir = os.getcwd()
print("Notebook directory: ", tutorial_home_dir)
# assumption is this is in the notebook_pges subdirectory of the repo
base_dir = os.path.abspath(os.path.join(tutorial_home_dir, '..'))

# directory for data downloads
output_dir = os.path.join(base_dir, 'notebook_output', 'topsApp')
slc_dir = os.path.join(output_dir, 'data', 'slcs')
orbit_dir = os.path.join(output_dir, 'data', 'orbits')
insar_dir = os.path.join(output_dir, 'insar')

# generate all the folders in case they do not exist yet
os.makedirs(slc_dir, exist_ok=True)
os.makedirs(orbit_dir, exist_ok=True)
os.makedirs(insar_dir, exist_ok=True)

# Always start at the notebook directory    
os.chdir(tutorial_home_dir)

## Setting up isce2 and endpoint data services

In [ ]:
import os
import json
from math import floor, ceil
import json
import re
import osaka
import osaka.main
from builtins import str
import os, sys, re, json, logging, traceback, requests, argparse
from datetime import datetime
from pprint import pformat

from requests.packages.urllib3.exceptions import (InsecureRequestWarning,
                                                  InsecurePlatformWarning)
import isce
from iscesys.Component.ProductManager import ProductManager as PM

try: from html.parser import HTMLParser
except: from html.parser import HTMLParser
try:
    sys.path.append(os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(__file__))), "python"))
except:
    sys.path.append(os.path.join(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))), "python"))
    
import topsApp_util

log_format = "[%(asctime)s: %(levelname)s/%(funcName)s] %(message)s"
logging.basicConfig(format=log_format, level=logging.INFO)
logger = logging.getLogger('create_ifg')

        
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
requests.packages.urllib3.disable_warnings(InsecurePlatformWarning)
PROCESSING_START=datetime.now().strftime("%Y-%m-%dT%H:%M:%S")

PGE_BASE=os.getcwd()

ISCE_HOME="/opt/isce2/isce"

SLC_RE = re.compile(r'(?P<mission>S1\w)_IW_SLC__.*?' +
                    r'_(?P<start_year>\d{4})(?P<start_month>\d{2})(?P<start_day>\d{2})' +
                    r'T(?P<start_hour>\d{2})(?P<start_min>\d{2})(?P<start_sec>\d{2})' +
                    r'_(?P<end_year>\d{4})(?P<end_month>\d{2})(?P<end_day>\d{2})' +
                    r'T(?P<end_hour>\d{2})(?P<end_min>\d{2})(?P<end_sec>\d{2})_.*$')


QC_SERVER = 'https://qc.sentinel1.eo.esa.int/'
DATA_SERVER = 'http://aux.sentinel1.eo.esa.int/'

ORBITMAP = [('precise','aux_poeorb', 100),
            ('restituted','aux_resorb', 100)]

OPER_RE = re.compile(r'S1\w_OPER_AUX_(?P<type>\w+)_OPOD_(?P<yr>\d{4})(?P<mo>\d{2})(?P<dy>\d{2})')

wd = os.getcwd()

if isinstance(reference_slcs, str):
    reference_slcs = json.loads(reference_slcs)
if isinstance(secondary_slcs, str):
    secondary_slcs = json.loads(secondary_slcs)
    
if isinstance(swaths, str):
    swaths = json.loads(swaths)
    
localize_slcs = reference_slcs + secondary_slcs
tops_properties = {}
tops_properties["swaths"] = swaths
tops_properties["range looks"] = range_looks
tops_properties["azimuth looks"] = azimuth_looks
tops_properties["do unwrap"] = do_unwrap
tops_properties["unwrapper name"] = unwrapper_name
tops_properties["do denseoffsets"] = do_denseoffsets
tops_properties["region of interest"] ="[{}, {}, {}, {}]".format(min_lat, max_lat, min_lon, max_lon)

input_dict = {}
input_dict["reference_slcs"] = reference_slcs
input_dict["secondary_slcs"] = secondary_slcs
input_dict["localize_slcs"] = localize_slcs
input_dict["min_lat"]=min_lat
input_dict["max_lat"]=max_lat
input_dict["min_lon"]=min_lon
input_dict["max_lon"]=max_lon


input_dict["sensor_name"]=sensor_name
input_dict["swaths"] = swaths
input_dict["range_looks"]=range_looks
input_dict["azimuth_looks"]=azimuth_looks
input_dict["do_unwrap"]=do_unwrap
input_dict["unwrapper_name"]=unwrapper_name
input_dict["do_denseoffsets"]=do_denseoffsets


wgs84_file = ''


# directory in which the notebook resides
if 'tutorial_home_dir' not in globals():
    tutorial_home_dir = os.getcwd()
print("Notebook directory: ", tutorial_home_dir)

# directory for data downloads
slc_dir = os.path.join(tutorial_home_dir,'data', 'slcs')
orbit_dir = os.path.join(tutorial_home_dir, 'data', 'orbits')
insar_dir = os.path.join(tutorial_home_dir, 'insar')


## 1. Collecting Input Datasets
 - Downnloading **SLCs**
 - Downloading **Orbits**
 - Downloading **Dems**
 

### 1.1 SLC Download

TOPS SLC product files delivered from ESA are zip archives. When unpacked the zip extension will be replaced by SAFE. The products are therefore also frequently called SAFE files. topsApp.py can read the data from either a zip file or a SAFE file. To limit disk usage, it is recommended to not unzip the individual files.

The zip or SAFE filenames provide information on the product type, the polarization, and the start and stop acquisition time. For example: S1A_IW_SLC__1SDV_20200511T135117_20200511T135144_032518_03C421_7768.zip
- Type = slc
- Polarization = Dual polarization
- Date = 20200511
- UTC time of acquisition = ~13:51
- Sensing start for the acquisition was 20200511 at 13:51:17


In [ ]:
topsApp_util.download_slcs(localize_slcs, slc_dir)
#extract_slc_data(slc_dir, localize_slcs)
! ls -lh {slc_dir}

### 1.2 Orbits Download

In addition to the **SAFE files**, **orbit files** and the **auxiliary instrument files** are required for ISCE processing. Both the orbit and instrument files are provided by ESA and can be downloaded at: https://qc.sentinel1.eo.esa.int/.

In [ ]:
from shutil import move
import glob

os.chdir(tutorial_home_dir)
print(tutorial_home_dir)

topsApp_util.get_orbit_files(localize_slcs)
# Move the orbits to orbit folder
orb_files = glob.glob("*.EOF")
for orb in orb_files:
    move(orb, os.path.join(orbit_dir, orb))
!ls {orbit_dir}

### 1.3 Dems Download

Dems over the region of intetrest is downloaded.

In [ ]:
import os

os.chdir(insar_dir)
topsApp_util.download_dem(min_lat, max_lat, min_lon, max_lon)

wgs84_file =''
if os.path.exists("dem.txt"):
    cmd = ["awk", "'/wgs84/ {print $NF;exit}'", "dem.txt"]
    WGS84 = topsApp_util.run_cmd_output(cmd).decode("utf-8").strip()
    wgs84_file = os.path.join(".", WGS84)
print(wgs84_file)

input_dict["wgs84_file"]=wgs84_file

### 1.4 AUX_CAL file download ####

The following calibration auxliary (AUX_CAL) file is used for **antenna pattern correction** to compensate the range phase offset of SAFE products with **IPF verison 002.36** (mainly for images acquired before March 2015). If all your SAFE products are from another IPF version, then no AUX files are needed. Check [ESA document](https://earth.esa.int/documents/247904/1653440/Sentinel-1-IPF_EAP_Phase_correction) for details. 

Run the command below to download the AUX_CAL file once and store it somewhere (_i.e._ ~/aux/aux_cal) so that you can use it all the time, for `stackSentinel.py -a` or `auxiliary data directory` in `topsApp.py`.

```
wget https://qc.sentinel1.eo.esa.int/product/S1A/AUX_CAL/20140908T000000/S1A_AUX_CAL_V20140908T000000_G20190626T100201.SAFE.TGZ
tar zxvf S1A_AUX_CAL_V20140908T000000_G20190626T100201.SAFE.TGZ
rm S1A_AUX_CAL_V20140908T000000_G20190626T100201.SAFE.TGZ

In [ ]:
%%bash
cd $insar_dir
mkdir -p ./AuxDir
wget https://qc.sentinel1.eo.esa.int/product/S1A/AUX_CAL/20140908T000000/S1A_AUX_CAL_V20140908T000000_G20190626T100201.SAFE.TGZ
tar zxvf S1A_AUX_CAL_V20140908T000000_G20190626T100201.SAFE.TGZ --directory ./AuxDir
rm S1A_AUX_CAL_V20140908T000000_G20190626T100201.SAFE.TGZ

## 2 topsApp Connfiguration Files


For topsApp processing, we use *topsApp.xml*, *reference.xml* and *secondary.xml*  as config files to send input data information to isce.

### 2.1 topsApp.xml

Example:
```xml
<?xml version="1.0" encoding="UTF-8"?>
<topsApp>
  <component name="topsinsar">
    <property name="Sensor name">SENTINEL1</property>
    <component name="reference">
        <catalog>reference.xml</catalog>
    </component>
    <component name="secondary">
        <catalog>secondary.xml</catalog>
    </component>
    <property name="swaths">[3]</property>
    <property name="range looks">7</property>
    <property name="azimuth looks">3</property>
    <property name="region of interest">[37.98, 38.33, -118.21, -117.68]</property>
    <property name="do unwrap">True</property>
    <property name="unwrapper name">snaphu_mcf</property>
    <property name="do denseoffsets">True</property>
    <property name="demfilename">path_to_your_dem</property>
    <property name="geocode demfilename">path_to_your_dem</property>
    <!--property name="geocode list">['merged/phsig.cor', 'merged/filt_topophase.unw', 'merged/los.rdr', 'merged/topophase.flat', 'merged/filt_topophase.flat','merged/topophase.cor','merged/filt_topophase.unw.conncomp']</property>-->
  </component>
</topsApp>
```

- The reference and secondary components refer  to their own *.xml* files 
- The **swaths** property controls the number of swaths to be processed. 
- **range looks** and **azimuth looks**: The range resolution for sentinel varies from near to far range, but is roughly 5m, while the azimuth resolution is approximately 15m, leading to a multi-looked product that will be approximately 35m by 45m.
- By specifying the **region of interest** as [S, N, W, E] to only capture the extent, topsApp.py will only extract those bursts from subswaths needed to cover the earthquake.
- By default, topsApp can download a DEM on the fly. By including **demFilename** a local DEM can be specified as input for the processing.
- By default, the geocoding in topsApp.py is performed at the same sampling as processing DEM. However, a different DEM *to be used specifically for geocoding* can be specified using the **geocode demfilename** property. This is used for the case when data has been multilooked to order of 100m or greater and when geocoding to 30m is an overkill.
- By default, no unwrapping is done. In order to turn it on, set the property **do unwrap** to *True*.
- In case unwrapping is requested, the default unwrapping strategy to be applied is the *icu* unwrapping method. For this tutorial, we will use *snaphu_mcf*.
- Lastly, we request topsApp.py to run the dense-offsets using the **do denseoffsets** property. By enabling this, topsApp.py will estimate the range and azimuth offsets on the amplitude of the reference and secondary SLC files.

In [ ]:
topsApp_util.create_topsApp_xml(tops_properties, input_dict)

### 2.2 reference.xml and secondary.xml

Example:
``` xml
<component name="reference">
    <property name="orbit directory">../data/orbits</property>
    <property name="output directory">reference</property>
    <property name="safe">['../data/slcs/S1A_IW_SLC__1SDV_20200511T135117_20200511T135144_032518_03C421_7768.zip']</property>
</component>
```
- The value associated with the reference **safe** property corresponds to a list of SAFE files that are to be mosaiced when generating the interferogram. 
- The **orbit directory** points  to the directory where we have stored the POEORB (precise) orbits for this example.

In [ ]:
xml_file = os.path.join(tutorial_home_dir, "support_docs/insar/reference.xml")
topsApp_util.create_xml(xml_file, 'reference', reference_slcs)

xml_file = os.path.join(tutorial_home_dir, "support_docs/insar/secondary.xml")
topsApp_util.create_xml(xml_file, 'secondary', secondary_slcs)

### 2.3 Moving input XML Files in Appropriate Directory

Moving the files in insar directory.

In [ ]:
## Template xml for this tutorial
from shutil import copyfile, move 

topsAppXml_original =  os.path.join(tutorial_home_dir,'support_docs/insar/topsApp.xml')  
refXml_original =  os.path.join(tutorial_home_dir,'support_docs/insar/reference.xml')  
secXml_original =  os.path.join(tutorial_home_dir,'support_docs/insar/secondary.xml')  

## Check if the topsApp.xml file already exists, if not copy the example for the excerisize
if not os.path.isfile(os.path.join(insar_dir,'topsApp.xml')):
    copyfile(topsAppXml_original, os.path.join(insar_dir, 'topsApp.xml'))
else:
    print(os.path.join(insar_dir,'topsApp.xml') + " already exist, will not overwrite")
    
if not os.path.isfile(os.path.join(insar_dir, 'reference.xml')):
    copyfile(refXml_original, os.path.join(insar_dir,'reference.xml'))
else:
    print(os.path.join(insar_dir,'reference.xml') + " already exist, will not overwrite")
    
if not os.path.isfile(os.path.join(insar_dir, 'secondary.xml')):
    copyfile(secXml_original,os.path.join(insar_dir, 'secondary.xml'))
else:
    print(os.path.join(insar_dir,'secondary.xml') + " already exist, will not overwrite")

## 3. topsApp.py processing steps

The topsApp.py workflow can be called with a single command-line call to topsApp.py; by default it will run all the required processing steps with inputs pulled from the topsApp.xml file. Although this is an attractive feature, it is recommended to run topsApp.py with “steps” enabled. This will allow you to re-start the processing from a given processing step. If “steps” are not used, users must restart processing from the beginning of the workflow after fixing any downstream issues with the processing.


**Steps of topsApp processing (in order)**:
 - startup
 - preproces
 - computeBaselines
 - verifyDEM
 - topo
 - subsetoverlaps
 - coarseoffsets
 - coarseresamp
 - overlapifg
 - prepesd
 - esd
 - rangecoreg
 - fineoffsets
 - fineresamp
 - ion
 - burstifg
 - mergebursts
 - filter
 - unwrap
 - unwrap2stage
 - geocode
 - denseoffsets
 - filteroffsets
 - geocodeoffsets
 


In [ ]:
!/opt/isce2/isce/applications/topsApp.py --help --steps

### 3.1 START THE PROCESSING
We will do processing from "startup" to "geocode" in one step in working directory (insar)

In [ ]:
os.chdir(insar_dir)
!/opt/isce2/isce/applications/topsApp.py --start=startup --end=geocode

### 3.2 PLOT  INTERFEROGRAM

Plotting filt_topophase.unw.geo with metaplotlib

In [ ]:
os.chdir(insar_dir)
topsApp_util.plot_wrapped_data_singleframe('merged/filt_topophase.flat.geo')

In [ ]:
import os
import folium
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import rasterio as rio
from rasterio.plot import show, plotting_extent
from rasterio.merge import merge
from PIL import Image, ImageChops

os.chdir(insar_dir)
src = rio.open('merged/filt_topophase.flat.geo')

fig, ax = plt.subplots(1, figsize=(18, 16))
data = src.read(1)
data[data==0] = np.nan
show(np.angle(data), cmap='rainbow', vmin=-np.pi, vmax=np.pi, transform=src.transform, ax=ax)
png_file = f'flat.png'
fig.savefig(png_file, transparent=True)

### 3.3 Generate the Product

In [ ]:
prod_name = topsApp_util.create_product(insar_dir, tops_properties, input_dict)
print(prod_name)

# Relevant references:
- Heresh Fattahi, Piyush Agram, and Mark Simons, *Precise coregistration of Sentinel-1A TOPS data*, https://files.scec.org/s3fs-public/0129_1400_1530_Fattahi.pdf
- Fattahi, H., Agram, P. and Simons, M., 2016. A network-based enhanced spectral diversity approach for TOPS time-series analysis. IEEE Transactions on Geoscience and Remote Sensing, 55(2), pp.777-786. https://core.ac.uk/reader/77927508
- ESA, *Sentinel-1 and TOPS overview*, https://sentinel.esa.int/web/sentinel/user-guides/sentinel-1-sar
- Nestor Yague-Martinez, Pau Prats-Iraola, Fernando Rodriguez Gonzalez,Ramon Brcic, Robert Shau, Dirk Geudtner, Michael Eineder, and Richard Bamler, *Interferometric Processing of Sentinel-1 TOPS Data*, IEEE, doi:10.1109/TGRS.2015.2497902, https://ieeexplore.ieee.org/document/7390052/
- Liang, C., Agram, P., Simons, M. and Fielding, E.J., 2019. Ionospheric correction of insar time series analysis of c-band sentinel-1 tops data. IEEE Transactions on Geoscience and Remote Sensing, 57(9), pp.6755-6773. 

<font size="1">This notebook is compatible with NISAR Jupyter Server Stack v1.4 and above</font>